## Setup environments

In [1]:
import os

from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm_model = 'gpt-4o-mini'
chat_model = ChatOpenAI(temperature=0.0, model=llm_model)

## Various Templates

In [11]:
biology_template = """BIOLOGY
You are a very smart biology professor. 
You are great at answering questions about biology in a concise and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""


math_template = """MATHEMATICS
You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

astronomy_template = """ASTRONOMY
ou are a very good astronomer. You are great at answering astronomy questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

travel_agent_template = """TRAVEL AGENT
You are a very good travel agent with a large amount
of knowledge when it comes to getting people the best deals and recommendations
for travel, vacations, flights and world's best destinations for vacation. 
You are great at answering travel, vacation, flights, transportation, tourist guides questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

default_prompt = """DEFAULT
please answer the following question:
{query}
"""

In [13]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings


prompt_templates = [biology_template, math_template, astronomy_template, travel_agent_template, default_prompt]

embeddings = OpenAIEmbeddings()
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [14]:
# 라우팅 함수 정의
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print(f"Using: {most_similar.split('\n')[0]}") # More descriptive printing
    return PromptTemplate.from_template(most_similar)

In [15]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | chat_model
    | StrOutputParser()
)

In [16]:
print(chain.invoke("What is photosynthesis?"))

Using: BIOLOGY
Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy in the form of glucose. This process primarily occurs in the chloroplasts of plant cells, where chlorophyll (the green pigment) captures light energy.

The overall equation for photosynthesis can be summarized as:

\[ 6 \, \text{CO}_2 + 6 \, \text{H}_2\text{O} + \text{light energy} \rightarrow \text{C}_6\text{H}_{12}\text{O}_6 + 6 \, \text{O}_2 \]

This means that carbon dioxide and water, in the presence of light, are transformed into glucose and oxygen. The glucose serves as an energy source for the plant, while the oxygen is released as a byproduct into the atmosphere.


In [17]:
print(chain.invoke("What are some good places to visit in Europe in the summer?"))

Using: TRAVEL AGENT
Europe in the summer offers a plethora of stunning destinations, each with its unique charm and attractions. Here’s a breakdown of some of the best places to visit, categorized by region:

### Western Europe
1. **Paris, France**
   - **Attractions**: Eiffel Tower, Louvre Museum, Montmartre.
   - **Activities**: Enjoy picnics in parks, river cruises on the Seine, and outdoor cafes.

2. **Amsterdam, Netherlands**
   - **Attractions**: Anne Frank House, Van Gogh Museum, canals.
   - **Activities**: Biking through the city, visiting flower markets, and enjoying open-air festivals.

3. **Barcelona, Spain**
   - **Attractions**: Sagrada Familia, Park Güell, Gothic Quarter.
   - **Activities**: Beach days at Barceloneta, tapas tours, and exploring vibrant street life.

### Southern Europe
4. **Santorini, Greece**
   - **Attractions**: Oia sunsets, volcanic beaches, ancient ruins.
   - **Activities**: Wine tasting, sailing tours, and exploring charming villages.

5. **Amalf

In [18]:
print(chain.invoke("Tell me about black holes."))

Using: ASTRONOMY
Certainly! To understand black holes, we can break the topic down into several key components: their definition, formation, properties, types, and significance in the universe.

### 1. Definition
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. This is why they are called "black" holes; they do not emit light and are invisible against the backdrop of space.

### 2. Formation
Black holes typically form from the remnants of massive stars after they have exhausted their nuclear fuel. Here’s a breakdown of the process:
- **Stellar Evolution**: A star spends most of its life fusing hydrogen into helium in its core. Once the hydrogen is depleted, the core contracts and heats up, allowing the star to fuse heavier elements.
- **Supernova**: For massive stars (typically more than 20 times the mass of the Sun), once iron builds up in the core, fusion stops, and the core collapses under gravity. This col

In [19]:
print(chain.invoke("What is the integral of x^2?"))

Using: MATHEMATICS
To find the integral of \( x^2 \), we can use the power rule for integration. The power rule states that:

\[
\int x^n \, dx = \frac{x^{n+1}}{n+1} + C
\]

where \( n \neq -1 \) and \( C \) is the constant of integration.

In this case, we have \( n = 2 \). Applying the power rule:

\[
\int x^2 \, dx = \frac{x^{2+1}}{2+1} + C = \frac{x^3}{3} + C
\]

So, the integral of \( x^2 \) is:

\[
\frac{x^3}{3} + C
\]


In [24]:
print(chain.invoke("Tell me a price for iPhone 15 Pro."))

Using: DEFAULT
As of my last update in October 2023, the starting price for the iPhone 15 Pro typically began around $999 in the United States for the base model. However, prices can vary based on storage options and regional differences. For the most accurate and current pricing, I recommend checking Apple's official website or authorized retailers.
